In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
import gzip
import os
import shutil
import pandas as pd
from sklearn.model_selection import train_test_split
import wget

# Data

## Download

In [3]:
URL = 'https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/covtype.data.gz'

In [5]:
if not os.path.exists('../data/covtype'):
    os.makedirs('../data/covtype')

In [6]:
filename = wget.download(URL)

100% [........................................................................] 11240707 / 11240707

In [8]:
with gzip.open(filename, 'rb') as f_in:
    with open('../data/covtype/covtype.csv', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

In [9]:
df = pd.read_csv('../data/covtype/covtype.csv')

In [10]:
n_total = len(df)
n_total

581011

In [11]:
train_val_indices, test_indices = train_test_split(range(n_total), test_size=0.2, random_state=0)
train_indices, val_indices = train_test_split(train_val_indices, test_size=0.2/0.6, random_state=0)

In [12]:
traindf = df.iloc[train_indices]
valdf = df.iloc[val_indices]
testdf = df.iloc[test_indices]

In [14]:
traindf = traindf.sample(frac=1)

In [15]:
traindf.to_csv('../data/covtype/train_covertype.csv', index=False, header=False)
valdf.to_csv('../data/covtype/val_covertype.csv', index=False, header=False)
testdf.to_csv('../data/covtype/test_covertype.csv', index=False, header=False)

## Data tensors

In [18]:
import numpy as np
import torch

In [19]:
SEED = 1

In [20]:
torch.manual_seed(SEED)
np.random.seed(SEED)

In [ ]:
import torch.nn.functional as F
import torch
from torch import nn
from torch.nn import init
from functools import partial
from torch.utils.data import TensorDataset
from dltools.callback import AvgStatsCallback, sched_lin, sched_cos, Recorder, AvgStatsCallback
from dltools.callback import ParamScheduler, combine_scheds, Callback, LearningrateFinder, DeviceCallback
from dltools.callback import BatchTransformXCallback
from dltools.databunch import DataBunch
from dltools.functions import create_learner, get_dls, get_model, get_model_func, listify, normalize_to, get_data
from dltools.functions import view_tfm, init_cnn, activation_stats
from dltools.hook import Hook, Hooks, ListContainer, children, append_stats, append_stats_hist, get_min, get_hist
from dltools.layer import Lambda, flatten, conv_layer, GeneralRelu
from dltools.metrics import accuracy
from dltools.runner import get_learn_run, get_cnn_model, get_cnn_layers, init_cnn, get_runner
from matplotlib import pyplot as plt

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else "cpu"
device

In [ ]:
# !type fastaimanual.py

In [ ]:
# changes rounding apparently
torch.set_num_threads(2)

In [ ]:
x_train, y_train, x_valid, y_valid = get_data()
x_train, x_valid = normalize_to(x_train, x_valid)
train_ds, valid_ds = TensorDataset(x_train, y_train), TensorDataset(x_valid, y_valid)

In [ ]:
x_train.mean(), x_train.std(), x_valid.mean(), x_valid.std()

In [ ]:
nh, bs = 50, 512
c = y_train.max().item()+1
loss_func = F.cross_entropy
data = DataBunch(*get_dls(train_ds, valid_ds, bs), c)

In [ ]:
# number of filters
nfs = [8, 16, 32, 64, 64]

In [ ]:
mnist_view = view_tfm(1, 28, 28)

In [ ]:
# cbfs = [LearningrateFinder, Recorder]
cbfs = [DeviceCallback, Recorder, partial(AvgStatsCallback, accuracy), partial(BatchTransformXCallback, mnist_view)]

### Hooks Class

In [ ]:
hooks = [Hook(l, append_stats) for l in children(model[:4])]

In [ ]:
run.fit(1, learn)

In [ ]:
for h in hooks:
    plt.plot(h.stats[0])
    h.remove()
plt.legend(range(4));

In [ ]:
sched = combine_scheds([0.5, 0.5], [sched_cos(0.2, 1.), sched_cos(1., 0.1)])

In [ ]:
# learn, run = get_learn_run(nfs, data, 1., conv_layer, cbs=cbfs+[partial(ParamScheduler, 'lr', sched)])

In [ ]:
# In the notebook the line is ... for l in children(model[:4]) because the model
# resides in the __main__ namespace, and therefore the hooks can be registered in the model there.
# However, since get_learn_run() is in dltools.runner, the model resides in dltools.runner, in the Learner
# object learn, and can only be accessed via learn.
# hooks = [Hook(l, append_stats) for l in children(model[:4])] # fastai notebook
hooks = [Hook(l, append_stats) for l in children(learn.model[:4])]

In [ ]:
run.fit(1, learn)

In [ ]:
for h in hooks:
    plt.plot(h.stats[0])
plt.legend(range(4))

In [ ]:
for h in hooks:
    print(h.stats)

In [ ]:
learn,run = get_learn_run(nfs, data, 1., conv_layer, uniform=True,
                          cbs=cbfs+[partial(ParamScheduler,'lr', sched)])

In [ ]:
run.fit(8, learn)

In [ ]:
with Hooks(model, append_stats) as hooks:
    run.fit(1, learn)
    fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(10, 4))
    for h in hooks:
        ms,ss = h.stats
        ax0.plot(ms[:10])
        ax1.plot(ss[:10])        
    plt.legend(range(5));
    
    fig,(ax0,ax1) = plt.subplots(1,2, figsize=(10,4))
    for h in hooks:
        ms,ss = h.stats
        ax0.plot(ms)
        ax1.plot(ss)
    plt.legend(range(5));

In [ ]:
run.fit(8, learn)

In [ ]:
with Hooks(model, append_stats) as hooks:
    run.fit(1, learn)
    fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(10, 4))
    for h in hooks:
        ms, ss = h.stats
        ax0.plot(ms[:10])
        ax1.plot(ss[:10])
    plt.legend(range(6))
    
    fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(10, 4))
    for h in hooks:
        ms, ss = h.stats
        ax0.plot(ms)
        ax1.plot(ss)
    plt.legend(range(6))

### Initialization and telemetry setup

#### Check initialization

In [ ]:
def get_cnn_layers_simple(data, nfs):
    nfs = [1] + nfs
    return [
        conv2d(nfs[i], nfs[i+1], 5 if i==0 else 3)
        for i in range(len(nfs)-1)
    ] + [nn.AdaptiveAvgPool2d(1), Lambda(flatten), nn.Linear(nfs[-1], data.c)]

def get_cnn_model_simple(data, nfs): return nn.Sequential(*get_cnn_layers_simple(data, nfs))

def conv2d(ni, nf, ks=3, stride=2):
    return nn.Sequential(
        nn.Conv2d(ni, nf, ks, padding=ks//2, stride=stride), nn.ReLU())

In [ ]:
model = get_cnn_model_simple(data, nfs)
learn, run = get_runner(model, data, lr=0.9, cbs=cbfs)

In [ ]:
def mnist_resize(x):
    return x.view(-1, 1, 28, 28)

In [ ]:
x, y = next(iter(data.train_dl))

In [ ]:
x = mnist_resize(x).cuda()

In [ ]:
# mean and std around 0, 1 from the normalization
x.mean(), x.std()

In [ ]:
# after the first layer, the mean and std are not close to 0 and 1 anymore
p = model.cuda()[0](x)
p.mean(), p.std()

In [ ]:
# initialize the model parameters with kaiming
for l in model.cuda():
    if isinstance(l, nn.Sequential):
        init.kaiming_normal_(l[0].weight)
        l[0].bias.data.zero_()

In [ ]:
# std is now close to 1, and mean is close to 0.5 due to relu cutoff
p = model.cuda()[0](x)
p.mean(), p.std()

#### Can also use get_cnn_model (with the conv_layer argument)

In [ ]:
model = get_cnn_model(data, nfs, conv_layer)
learn, run = get_runner(model, data, lr=0.9, cbs=cbfs)

In [ ]:
def mnist_resize(x):
    return x.view(-1, 1, 28, 28)

In [ ]:
x, y = next(iter(data.train_dl))

In [ ]:
x = mnist_resize(x)

In [ ]:
# mean and std around 0, 1 from the normalization
x.mean(), x.std()

In [ ]:
# after the first layer, the mean and std are not close to 0 and 1 anymore
p = model[0](x)
p.mean(), p.std()

In [ ]:
# initialize the model parameters with kaiming
for l in model:
    if isinstance(l, nn.Sequential):
        init.kaiming_normal_(l[0].weight)
        l[0].bias.data.zero_()

In [ ]:
# std is now close to 1, and mean is close to 0.5 due to relu cutoff
p = model[0](x)
p.mean(), p.std()

In [ ]:
p1 = model[1](model[0](x))

In [ ]:
p1.mean(), p1.std()

In [ ]:
data.train_dl.batch_size

In [ ]:
def conv2d(ni, nf, ks=3, stride=2):
    return nn.Sequential(
        nn.Conv2d(ni, nf, ks, padding=ks//2, stride=stride), nn.ReLU())

In [ ]:
def get_hist(h):
    return torch.stack(h.stats[2]).t().float().log1p()

In [ ]:
def get_min(h):
    # Assume first two bins of the histogram contain values close to 0.
    # Show relative amount of data in these bins compared to all bins.
    # This gives a ratio of values close to 0.
    h1 = torch.stack(h.stats[2]).t().float()
    return h1[:2].sum(0)/h1.sum(0)

In [ ]:
model = get_cnn_model_simple(data, nfs)

In [ ]:
learn, run = get_runner(model, data, lr=0.9, cbs=cbfs)

In [ ]:
for l in model:
    if isinstance(l, nn.Sequential):
        init.kaiming_normal_(l[0].weight)
        l[0].bias.data.zero_()

In [ ]:
# init_cnn(model, uniform=True)

In [ ]:
with Hooks(model, append_stats_hist) as hooks:
    run.fit(1, learn)

In [ ]:
fig,axes = plt.subplots(2,2, figsize=(15,6))
for ax,h in zip(axes.flatten(), hooks[:4]):
    ax.imshow(get_hist(h), origin='lower')
    ax.axis('off')
plt.tight_layout()

In [ ]:
fig,axes = plt.subplots(2,2, figsize=(15,6))
for ax,h in zip(axes.flatten(), hooks[:4]):
    ax.plot([0, 110], [1., 1.], c='r', linewidth=1)
    ax.plot(get_min(h))
    ax.set_ylim(0,1.1)
plt.tight_layout()

In [ ]:
with Hooks(model, append_stats_hist) as hooks:
    telemetrize()

### Telemetry

#### Telemetry start

In [ ]:
from dltools.hook import telemetrize

In [ ]:
model = get_cnn_model(data, nfs, conv_layer)
learn, run = get_runner(model, data, lr=0.9, cbs=cbfs)

In [ ]:
init_cnn(model, True)

In [ ]:
model

In [ ]:
activation_stats(model, data, [-1, 1, 28, 28])

In [ ]:
learn.data.train_ds.tensors[0].shape[0]

In [ ]:
len(learn.data.train_ds)

In [ ]:
with Hooks(model, append_stats_hist) as hooks:
    run.fit(1, learn)
    telemetrize(hooks, xlim=1*len(learn.data.train_ds)//bs)

#### Telemetry for model with generalized relu

In [ ]:
from dltools.runner import get_cnn_grelu_model
from dltools.hook import telemetrize

In [ ]:
leak = [0.1, 0.1, 0.1, 0.1, 0.1]
clamp = [0.4, 0.2, 0.4, 0.2, 0.2]
model = get_cnn_grelu_model(data, nfs, leak, clamp, conv_layer)

In [ ]:
model

In [ ]:
learn, run = get_runner(model, data, lr=0.5, cbs=cbfs)

In [ ]:
init_cnn(model, True)

In [ ]:
activation_stats(model, data, [-1, 1, 28, 28])

In [ ]:
with Hooks(model, append_stats_hist) as hooks:
    run.fit(2, learn)
    telemetrize(hooks, 2*len(learn.data.train_ds)//bs)

### Train

In [ ]:
model =  get_cnn_model(data, nfs, conv_layer, leak=0.1, sub=0.4, maxv=6.)

In [ ]:
layer_stats(model, data)

In [ ]:
init_cnn(model)

In [ ]:
layer_stats(model, data)

In [ ]:
learn,run = get_runner(model, data, lr=0.9, cbs=cbfs)

In [ ]:
with Hooks(model, append_stats_hist) as hooks:
    run.fit(1, learn)
    fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(10, 4))
    for h in hooks:
        ms, ss, hi = h.stats
        ax0.plot(ms[:10])
        ax1.plot(ss[:10])
        h.remove()
    plt.legend(range(5));
    
    fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(10, 4))
    for h in hooks:
        ms, ss, hi = h.stats
        ax0.plot(ms)
        ax1.plot(ss)
    plt.legend(range(5));

In [ ]:
learn.data

In [ ]:
learn.opt.zero_grad()

In [ ]:
sched = combine_scheds([0.5, 0.5], [sched_cos(0.2, 1.), sched_cos(1., 0.1)])

In [ ]:
learn, run = get_learn_run(nfs, data, lr=1., layer=conv_layer, cbs=cbfs + [partial(ParamScheduler, 'lr', sched)])

In [ ]:
run.fit(8, learn)